# Bayesian Optimization: Newspaper Vendor Problem

This problem is a "textbook example" Operations Research problem called the newsvendor problem. Also called "Single period model".


If we know:
1. The price the newsvendor buys the newspapers for
2. The price the newsvendor sells the newspapers for
3. The demand, or number of papers sold, that day
4. The decision: The number of newspapers the newsvendor decides to buy

we can directly calculate the newsvendor's profit. However, we don't know actual future demand.

The question is how to **optimize profit using limited information on actual demand**. We will use Bayesian Optimization to address this.


**Credits**: *This notebook is based on a PyData Global 2020 talk by Ravin Kumar, see: https://github.com/canyon289/PyDataGlobal_2020*

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats, optimize
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt

## Define objective function

In [ ]:
def daily_profit(inventory, demand, newsvendor_cost=5, customer_price=7):
    """Calculates profit for a given day given inventory and demand"""
    return customer_price*np.min([inventory, demand]) - newsvendor_cost*inventory


def objective(inventory: int, demands: iter, **kwargs):
    """Takes an iterable of demand values and returns (negative) the total profit"""
    # Use negative total profit so that the objective function can be minimized later
    objective_function_value = -np.sum([daily_profit(inventory=inventory, demand=d, **kwargs) for d in demands])
    return objective_function_value

**Example**: with static daily inventory value of 42 newspapers, given costs, sales prices and daily demands, the newsvendor obtains a total profit of:

In [ ]:
total_profit = -objective(inventory=42, demands=[38,47,42], newsvendor_cost=4, customer_price=7)

## Sample data

In [ ]:
# Generate train/test data for demand from a Normal distribution with given mu, sigma
mu = 40
sigma = 20

np.random.seed(seed=1234)
demand = np.round(stats.norm(mu, sigma).rvs(15)) 
demand_seen, demand_unseen = demand[:5], demand[5:]

Demands seen historically:

In [ ]:
demand_seen

## Bayesian model for demand

Using PyMC3

In [ ]:
with pm.Model() as newsvendor:
    
    # Define priors
    sd = pm.HalfStudentT("standard_deviation_of_newspaper_demand", sigma=10, nu=20)
    mu = pm.Normal("mean_of_newspaper_demand", demand_seen.mean(), 20)
    demand = pm.TruncatedNormal("demand", mu=mu, sd=sd, lower=0, observed = demand_seen)

    # Sample posterior
    trace = pm.sample(tune=5000, draws=10000, chains=2)
    posterior_predictive = pm.sample_posterior_predictive(trace, progressbar=False)
    inf_data = az.from_pymc3(trace=trace, posterior_predictive=posterior_predictive)

## Numeric Optimization

Using scipy optimizer

In [ ]:
bayesian_demand_estimates = inf_data.posterior_predictive["demand"].values.flatten()
opt_stoch = optimize.minimize_scalar(objective, bounds=(0, np.inf), args=(bayesian_demand_estimates,))
opt_stoch

In [ ]:
f"Optimal inventory from Bayesian demand estimation and optimizer is {np.round(opt_stoch.x)} newspapers"

## Performance on unseen data

In [ ]:
profit = -objective(inventory=np.round(opt_stoch.x), demands=demand_unseen)
n_days = len(demand_unseen)

"Testing inventory choice over {:n} unseen days yields a total profit of {:.2f}, or {:.2f} per day." \
    .format(n_days, profit, profit/n_days)